In [ ]:
import numpy as np
from yahooquery import Ticker
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import investpy as inv
etfs=inv.get_etfs()
desconsiderados=['currency', 'bond']
etfs=etfs[~etfs.asset_class.isin(desconsiderados)]
ativos_americanos=etfs[etfs['country']=='united states']
lista=list(ativos_americanos['symbol'])
pd.set_option('display.max_rows', None)

In [ ]:
def clean(tic_list):
    words=tic_list.split()
    linha=[None]*len(words)
    coluna=[None]*len(words)
    coluna[0]="ETF"
    linha[0]='{}'.format(words[0].replace('dict_items([(','').replace(',','').replace("'","").replace('"',"").replace(')','').replace('(',''))
    words_limpo=words
    c=1
    l=1
    for i in range(1, len(words)):
      if ":" in words_limpo[i]:
        coluna[c]='{}'.format(words_limpo[i].replace(',','').replace('\t', '').replace("'","").replace('"',"").replace('{',"").replace(':',"").replace('[',"").replace(')','').replace('(',''))
        c+=1
      else:
        if c-l==1:
          
          linha[l]='{}'.format(words_limpo[i].replace(',','').replace('\t', '').replace('}','').replace(']','').replace(')','').replace('(',''))
          l+=1
        else:
          while c-l!=1:
            linha[l]=0
            l+=1  
          linha[l]='{}'.format(words_limpo[i].replace(',','').replace('\t', '').replace('}','').replace(']','').replace(')','').replace('(',''))
          l+=1
    valores=filter(None.__ne__,coluna)
    coluna=list(valores)
    valores=filter(None.__ne__,linha)
    linha=list(valores)

    take='_performanceOverview'
    for n in range (2, len(coluna)):
        if coluna[n]=='performanceOverview':
            take='performanceOverview'
        
        if coluna[n]=='performanceOverviewCat':
            take='performanceOverviewCat'
            
        if coluna[n]=='trailingReturns':
            take='trailingReturns'
            
        if coluna[n]=='trailingReturnsNav':
            take='trailingReturnsNav'
        
        if coluna[n]=='trailingReturnsCat':
            take='trailingReturnsCat'
        
        if coluna[n]=='annualTotalReturns':
            take=''
            take_aux='annualTotalReturns'
            
        if coluna[n]=='returnsCat':
            take=''
            take_aux='returnsCat'
            
        if coluna[n]=='riskOverviewStatistics':
            take=''
            take_aux='riskOverviewStatistics'
            
        if coluna[n]=='riskOverviewStatisticsCat':
            take=''
            take_aux='riskOverviewStatisticsCat'
        
        
        if coluna[n]=='year':
            temp=linha[n]
            temp=temp.replace("'","")
            take=take_aux+'_'+temp
        
   
        
        if coluna[n]!= take and coluna[n]!='year':
            coluna[n]=coluna[n]+'_'+take
   

    arr1=pd.DataFrame([coluna,linha],columns=coluna)
    arr1.drop([0], inplace=True)
    
    
    arr1.drop(columns=['year','returnsCat_', 'returns_', '210000_performanceOverview', '210000_trailingReturns', 'performanceOverview', 'trailingReturns', 'trailingReturnsNav','annualTotalReturns_', 'trailingReturnsCat' ], inplace=True)
    arr1.replace('[', np.nan, inplace=True)
    return arr1

    

In [ ]:
lista_etfs=lista

remove=['GRI']

for i in range (len(lista_etfs)):
    if lista_etfs[i] not in remove:
        fund = Ticker(lista_etfs[i])
        tic=fund.fund_performance
        tic_list=(str(tic.items()))

        if len(tic_list)>1000:
            print(i,lista_etfs[i], type(tic_list), len(tic_list))
            if i==0: 
                etf_performance=clean(tic_list)
                etf_performance = etf_performance.loc[:,~etf_performance.columns.duplicated()]
            else:
                arr1=clean(tic_list)
                arr1 = arr1.loc[:,~arr1.columns.duplicated()]
                etf_performance=pd.concat([etf_performance,arr1])
    

etf_performance.to_excel('etf_performance.xlsx')
etf_performance.transpose().to_excel('etf_performance_tranpose.xlsx')


